# Chapter3. RNN
### (1) Code run Through (hihello)
---

In [149]:
def make_chars(s) :
    return 
def make_character_dictionary(chars) :
    return {c:i for i,c in enumerate(chars)}
def count_isize(char_dictionary) :
    return len(char_dictionary)
# ==================================================================================================================
sentence = 'hihello'
char_set = list(set(sentence))
c2i = {c : i for i, c in enumerate(char_set)}
i2c = {i : c for i, c in enumerate(char_set)}
input_size = len(char_set)
hidden_size = len(char_set)
learning_rate = 0.1

x_data = [[c2i[x] for x in sentence[:-1]]]
x_one_hot = []
for x in sentence[:-1] :
    v = np.zeros(input_size, dtype=int)
    v[c2i[x]] = 1
    x_one_hot.append(list(v))
x_one_hot = [x_one_hot]
y_data = [[c2i[y] for y in sentence[1:]]]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

### (2) Code run through (charseq)

In [153]:
sample = ' if they want me'
char_set = list(set(sample))
char_dic = {c : i for i, c in enumerate(char_set)}
i2c = {i : c for i, c in enumerate(char_set)}
dic_size = len(char_dic)
hidden_size = dic_size
learning_rate = 0.1
# ==================================================================================================================
sample_idx = [char_dic[c] for c in sample]
x_one_hot = [[np.eye(dic_size)[x] for x in sample_idx[:-1]]]
y_data = [sample_idx[1:]]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

### (3) Code run through
---

In [158]:
input_size = dic_size
rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

for i in range(100):
    optimizer.zero_grad()
    outputs, _status = rnn(X)
    print(outputs[:,-1,:])
    print()
    print(_status)
    break

tensor([[ 0.4344,  0.2366, -0.0411, -0.6143, -0.2741, -0.0915, -0.3048,  0.2701,
          0.3302, -0.2971,  0.2413]], grad_fn=<SliceBackward0>)

tensor([[[ 0.4344,  0.2366, -0.0411, -0.6143, -0.2741, -0.0915, -0.3048,
           0.2701,  0.3302, -0.2971,  0.2413]]], grad_fn=<StackBackward0>)


In [259]:
rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)
epochs = 1
for epoch in range(epochs) :
    optimizer.zero_grad()
    hidden_state, output = rnn(X)
    print(rnn)
    print('=====================================================================')
    print(rnn.forward)

    # (10,15) & (1,15)
    # ======================================================================================= 현재 예측값 찍어주기
    # Torch 를 numpy 로 가져오고 싶으면 반드시 .data 를 해주어야 한다.
    predict_sequence = [i2c[int(p)] for p in torch.argmax(hidden_state.view(-1,input_size), dim=1)]    
    print(''.join(predict_sequence))
    # criterion 은 CrossEntropy 로써  real Y 하나와 LogProbability 곱하고 전체적으로 음수화 시킨다.
    # =========================== Criterion 의 역할
    # 1. Loss 를 구한다. Loss 은 단순한 scalar 이 아니다. 
    # grad_fn=<NllLossBackward0> 을 가지고 있어서, backward 라는 method 를 가지고 있다.
    # 이 method 를 이용해서 각 parameter 에 뿌려준다. ( 필히 frame work 를 써야 되는 이유)
    # 만약에 순수하게 값만 보고 싶다면 .item() 을 써준다.
    loss = criterion(hidden_state.view(-1,input_size), Y.view(-1))
    loss.backward()
    # ========================== 옵티마이저는 파라미터를 변수로 가지고 있다.
    # 옵티마이저의 수행 순서
    # 1. Gradient 를 받아먹고
    # 2. Parameter 들을 Update 시킨다.
    optimizer.step()  
    

RNN(11, 11, batch_first=True)
<bound method RNNBase.forward of RNN(11, 11, batch_first=True)>
nennfenntaefnnn


# Chapter4. Long Sequence
### (1) Making sequence dataset from long sentence (code)
---

In [180]:
x_data = []
y_data = []
sentence = ("if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.")
# ==================================================================================================================
# Char dict 에는 부호랑 띄어씌기도 포함된다.
chars         = set([s for s in sentence])
char_dic = {c:i for i, c in enumerate(chars)}
dic_size = len(char_dic)
hidden_size = dic_size 
sequence_length = 10
for i in range(0,len(sentence)-sequence_length) :
    x_str = sentence[i:i+sequence_length]
    y_str = sentence[i+1:i+sequence_length+1]
    print(i, x_str, '->', y_str, end = '\n\n')
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

0 if you wan -> f you want

1 f you want ->  you want 

2  you want  -> you want t

3 you want t -> ou want to

4 ou want to -> u want to 

5 u want to  ->  want to b

6  want to b -> want to bu

7 want to bu -> ant to bui

8 ant to bui -> nt to buil

9 nt to buil -> t to build

10 t to build ->  to build 

11  to build  -> to build a

12 to build a -> o build a 

13 o build a  ->  build a s

14  build a s -> build a sh

15 build a sh -> uild a shi

16 uild a shi -> ild a ship

17 ild a ship -> ld a ship,

18 ld a ship, -> d a ship, 

19 d a ship,  ->  a ship, d

20  a ship, d -> a ship, do

21 a ship, do ->  ship, don

22  ship, don -> ship, don'

23 ship, don' -> hip, don't

24 hip, don't -> ip, don't 

25 ip, don't  -> p, don't d

26 p, don't d -> , don't dr

27 , don't dr ->  don't dru

28  don't dru -> don't drum

29 don't drum -> on't drum 

30 on't drum  -> n't drum u

31 n't drum u -> 't drum up

32 't drum up -> t drum up 

33 t drum up  ->  drum up p

34  drum up p -> drum up

### (2) Adding FC layer and stacking RNN

In [181]:
class Net(torch.nn.Module) :
    def __init__(self, input_dim, hidden_dim, layers) :
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers,batch_first = True)
        self.fc = nn.Linear(hidden_dim,hidden_dim, bias=True)
    def forward(self, x) :
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
net = Net(dic_size, hidden_size, 2)

In [138]:
criterion = nn.CrossEntropyLoss()
learning_rate=0.01
optimizer = optim.Adam(net.parameters(), learning_rate)
n_epochs = 100
for epoch in range(n_epochs) :
    optimizer.zero_grad()
    outputs = net(X)
    # ====================================================================================================
    # 일단 길게 가져가기!
    loss = criterion(outputs.view(-1,dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

# Chapter6. Seq2Seq
## (1) Apply Seq2Seq : Data Preprocessing
---

In [221]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
#========================================================================================================================
torch.manual_seed(0)
raw = ['I feel hungry.	나는 배가 고프다.',
       'Pytorch is very easy.	파이토치는 매우 쉽다.',
       'Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.',
       'Pytorch is very clear to use.	파이토치는 사용하기에 매우 직관적이다.']
SOS_token = 0
EOS_token = 1
#========================================================================================================================
class Vocab() :
    def __init__(self) :
        self.vocabs = set()
        self.vocab2index = {}
        self.n_vocab = 0
    def add_vocab(self, sentence) :
        tokens = sentence.split(" ")
        for token in tokens :
            self.vocabs.add(token)
        self.vocab2index = {c:i+2 for i,c in enumerate(self.vocabs)}
        self.vocab2index["<SOS>"] = 0
        self.vocab2index["<EOS>"] = 1 
        self.n_vocab =  len(self.vocabs)
        return self.vocabs
#========================================================================================================================
def filter_pair(pair, source_max_length, target_max_length):
    return len(pair[0].split(" ")) < source_max_length and len(pair[1].split(" ")) < target_max_length
#========================================================================================================================
# corpus = Document
def preprocess (corpus, source_max_length, target_max_length) :
    # =========================================================================
    # 일단 모든 문서를 읽는다.
    print("reading corpus...")
    pairs = []
    for line in corpus :
        # line 은 영어와 한글 pair 로 이뤄져 있다.
        pairs.append([s for s in line.strip().lower().split('\t')])
    # =========================================================================
    # 이 중에서 길이에 맞는 문장만 필터링 한다. 
    pairs = [pair for pair in pairs if filter_pair(pair, source_max_length, target_max_length)]
    print("Read {} sentence pairs".format(len(pairs)))
    # =========================================================================
    # 각 문장에서의 단어들을 잡는다.
    print("Counting words...")
    source_vocab = Vocab()
    target_vocab = Vocab()
    for pair in pairs :
        source_vocab.add_vocab(pair[0])
        target_vocab.add_vocab(pair[1])        
    print("source vobac size : ", source_vocab.n_vocab)
    print("target vobac size : ", target_vocab.n_vocab)
    # =========================================================================
    return pairs, source_vocab, target_vocab
#================================================================================================================
source_max_length = 10
target_max_length = 12
pairs, source_vocab, target_vocab = preprocess (raw, source_max_length,target_max_length)

reading corpus...
Read 4 sentence pairs
Counting words...
source vobac size :  15
target vobac size :  11


In [227]:
source_vocab.vocabs

{'a',
 'clear',
 'deep',
 'easy.',
 'feel',
 'for',
 'framework',
 'hungry.',
 'i',
 'is',
 'learning.',
 'pytorch',
 'to',
 'use.',
 'very'}

## (2) Neural Net Setting
---

In [250]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        # Embedding 은 Setting 을 시켜주어야 한다.
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
    def forward(self, x, hidden):
        # 원하는 샘플만 16차원이 되었다.
        # 나머지는 다 동일하나, 차수만 하나 높여준 격이다.
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        return x, hidden

# ======================================================================================================================================================
class Decoder(nn.Module): # Decoder 도 분류이다.
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        x = self.softmax(self.out(x[0]))
        return x, hidden
    
hidden_size = 16
# 이 모델 안에서는 모든게 16차원이다.
encoder = Encoder(source_vocab.n_vocab,hidden_size)
decoder = Decoder(target_vocab.n_vocab,hidden_size)

In [254]:
a = X[0].view(-1,1)

In [255]:
encoder_hidden = torch.zeros([1, 1, 16])

encoder(a,encoder_hidden)

(tensor([[[-0.3810,  0.0866, -0.3435, -0.1414,  0.0611,  0.2244, -0.2186,
           -0.5115,  0.1847,  0.1543,  0.1257,  0.1222, -0.4421,  0.1064,
            0.0232,  0.1978]]], grad_fn=<StackBackward0>),
 tensor([[[-0.3810,  0.0866, -0.3435, -0.1414,  0.0611,  0.2244, -0.2186,
           -0.5115,  0.1847,  0.1543,  0.1257,  0.1222, -0.4421,  0.1064,
            0.0232,  0.1978]]], grad_fn=<StackBackward0>))

### (3) Training
---

In [237]:
def tensorize(vocab, sentence):
    indexes = [vocab.vocab2index[word] for word in sentence.lower().split(" ")]
    indexes.append(vocab.vocab2index["<EOS>"])
    return torch.LongTensor(indexes).view(-1, 1)

def train(pairs, source_vocab, target_vocab, encoder, decoder, n_iter, print_every=1000, learning_rate=0.01):
    for i in range(1, n_iter + 1):
        source_tensor = training_source[i - 1]
        target_tensor = training_target[i - 1]
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)
        # ====================================================================================
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        # 아마도 seq 길이를 구하는 것일 것이다.
        source_length = source_tensor.size(0)
        target_length = target_tensor.size(0)
        loss = 0
        # 한문장씩 넣어준다.
        for enc_input in range(source_length):
            _, encoder_hidden = encoder(source_tensor[enc_input], encoder_hidden)

In [103]:
def train(pairs, source_vocab, target_vocab, encoder, decoder, n_iter, print_every=1000, learning_rate=0.01):
 decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
 decoder_hidden = encoder_hidden
 for di in range(target_length):
 decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
 loss += criterion(decoder_output, target_tensor[di])
 decoder_input = target_tensor[di] # teacher forcing
 loss.backward()
 encoder_optimizer.step()
 decoder_optimizer.step()
 loss_iter = loss.item() / target_length
 loss_total += loss_iter
 if i % print_every == 0:
 loss_avg = loss_total / print_every
 loss_total = 0
 print("[{} - {}%] loss = {:05.4f}".format(i, i / n_iter * 100, loss_avg))